In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, FloatType
from pyspark.sql.functions import *
import random
from pyspark.sql import Row
# Criando uma sessão Spark
spark = SparkSession.builder.appName("SQL").getOrCreate()

# Criando DataFrame de clientes
schema_clientes = StructType([
    StructField('cliente_id', IntegerType(), True),
    StructField('nome', StringType(), True),
    StructField('idade', IntegerType(), True),
    StructField('cidade', StringType(), True),
    StructField('genero', StringType(), True)
])

clientes_data = [
    (i, 'Cliente' + str(i), random.randint(18, 65),
     random.choice(['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Brasília']),
     random.choice(['M', 'F']))
    for i in range(1, 1000000)
]

df_clientes = spark.createDataFrame(clientes_data, schema=schema_clientes)

# Criando DataFrame de produtos
schema_produtos = StructType([
    StructField('produto_id', IntegerType(), True),
    StructField('nome_produto', StringType(), True),
    StructField('categoria', StringType(), True),
    StructField('preco', FloatType(), True)
   
])

produtos_data = [
    Row(
        produto_id=i,
        nome_produto='Produto' + str(i),
        categoria=random.choice(['Eletrônicos', 'Roupas', 'Alimentos', 'Acessórios']),
        preco=random.uniform(10, 1000),  # Convertendo para float
        disponibilidade=random.choice([True, False])
    )
    for i in range(1, 1000000)
]

# Criando o DataFrame de produtos
df_produtos = spark.createDataFrame(produtos_data)

# # Criando DataFrame de vendas
schema_vendas = StructType([
    StructField('venda_id', IntegerType(), True),
    StructField('cliente_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('quantidade', IntegerType(), True),
    StructField('data_venda', StringType(), True)
])

vendas_data = [
    (i, random.randint(1, 100), random.randint(1, 50), random.randint(1, 5), '2023-01-01')
    for i in range(1, 1000000)
]

df_vendas = spark.createDataFrame(vendas_data, schema=schema_vendas)

# Criando DataFrame de transações
schema_transacoes = StructType([
    StructField('transacao_id', IntegerType(), True),
    StructField('produto_id', IntegerType(), True),
    StructField('valor', FloatType(), True),
    StructField('tipo', StringType(), True),
    StructField('status', StringType(), True)
])

transacoes_data = [
    (i, random.randint(1, 50), random.uniform(50, 500),
     random.choice(['Débito', 'Crédito']), random.choice(['Aprovado', 'Pendente', 'Rejeitado']))
    for i in range(1, 1000000)
]

df_transacoes = spark.createDataFrame(transacoes_data, schema=schema_transacoes)

23/12/09 12:16:29 WARN Utils: Your hostname, DESKTOP-0KANSSR resolves to a loopback address: 127.0.1.1; using 172.18.127.6 instead (on interface eth0)
23/12/09 12:16:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/12/09 12:16:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/09 12:16:31 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Criando visões temporárias para os DataFrames
# df_clientes.createOrReplaceTempView("clientes_view")
# df_vendas.createOrReplaceTempView("vendas_view")
# df_produtos.createOrReplaceTempView("produtos_view")
# df_transacoes.createOrReplaceTempView("transacoes_view")

In [3]:
# query = """
#     SELECT c.nome AS cliente,t.status,
#        COUNT(v.venda_id) AS total_vendas,
#        AVG(p.preco) AS media_preco,
#        COUNT(t.transacao_id) AS total_transacoes
# FROM clientes_view c
# INNER JOIN vendas_view v ON c.cliente_id = v.cliente_id
# LEFT JOIN produtos_view p ON v.produto_id = p.produto_id
# LEFT JOIN transacoes_view t ON p.produto_id = t.produto_id
# WHERE p.disponibilidade = True
#   AND (t.status NOT IN ('Aprovado') OR t.status IS NULL)
# GROUP BY c.nome,t.status
# HAVING AVG(p.preco) > 400
#    AND COUNT(t.transacao_id) > 3;

# """
# result = spark.sql(query)
# result.show()


In [2]:
df_produtos = df_produtos.filter(col("disponibilidade")==True)
df_transacoes = df_transacoes.filter("status not in ('Aprovado')")

In [3]:
df_parcial = df_clientes.alias("c").join(df_vendas.alias("v"), on=col("c.cliente_id") == col("v.cliente_id"))\
    .join(df_produtos.alias('p'), col("v.produto_id")==col("p.produto_id"))\
    .join(df_transacoes.alias('t'), col("p.produto_id")==col("t.produto_id"))\
    .select("c.nome", "v.venda_id", "p.preco", "t.transacao_id", "t.status")

In [4]:
df_parcial.groupBy('nome', 'status')\
    .agg(
        count('venda_id').alias("total_vendas"),
        avg("preco").alias("media_preco"),
        count("transacao_id").alias("total_transacoes")
    )\
    .filter("total_transacoes > 3 and media_preco > 400")\
    .show()

23/12/09 12:18:36 WARN TaskSetManager: Stage 0 contains a task of very large size (2220 KiB). The maximum recommended task size is 1000 KiB.
23/12/09 12:18:37 WARN TaskSetManager: Stage 1 contains a task of very large size (1175 KiB). The maximum recommended task size is 1000 KiB.
23/12/09 12:18:38 WARN TaskSetManager: Stage 2 contains a task of very large size (2705 KiB). The maximum recommended task size is 1000 KiB.
23/12/09 12:18:38 WARN TaskSetManager: Stage 3 contains a task of very large size (1744 KiB). The maximum recommended task size is 1000 KiB.


+---------+---------+------------+-----------------+----------------+
|     nome|   status|total_vendas|      media_preco|total_transacoes|
+---------+---------+------------+-----------------+----------------+
|Cliente82|Rejeitado|    32722088|560.5068435651759|        32722088|
|Cliente99|Rejeitado|    32724014| 562.816994260191|        32724014|
|Cliente78|Rejeitado|    33766132|565.6189784591949|        33766132|
| Cliente3|Rejeitado|    33709816|558.2693993277513|        33709816|
|Cliente76|Rejeitado|    33614004|556.3282497761035|        33614004|
| Cliente4|Rejeitado|    32389219|562.6898640081893|        32389219|
|Cliente82| Pendente|    32694897|560.9967612305907|        32694897|
|Cliente48|Rejeitado|    33255384| 552.383472784225|        33255384|
|Cliente94|Rejeitado|    33315869| 561.710608960709|        33315869|
|Cliente19| Pendente|    33410157|557.4745131439763|        33410157|
|Cliente61|Rejeitado|    33720791|555.2121831254788|        33720791|
|Cliente89|Rejeitado